In [1]:
library(TCGAbiolinks)
library(DESeq2)
library(ggplot2)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: MatrixGe

In [2]:
#download
# query.KIRP <- GDCquery(project = "TCGA-KIRP", 
#                        data.category = "Transcriptome Profiling", 
#                        data.type = "Gene Expression Quantification", 
#                        workflow.type = "HTSeq - Counts", 
#                        sample.type = "Primary Tumor")

# GDCdownload(query.KIRP)
# tumor.kirp <- GDCprepare(query = query.KIRP, 
#                          save = TRUE, 
#                          save.filename = "kirpRNAseqTumor.rda", 
#                          summarizedExperiment = TRUE)

In [3]:
load("../../../Data/kirpRNAseqTumor.rda")
tumor.kirp = data

In [4]:
#download
#query.KIRP_normal <- GDCquery(project = "TCGA-KIRP", 
                       #data.category = "Transcriptome Profiling", 
                       #data.type = "Gene Expression Quantification", 
                       #workflow.type = "HTSeq - Counts", 
                       #sample.type = "Solid Tissue Normal")

#GDCdownload(query.KIRP_normal)
#tumor.kirp_normal <- GDCprepare(query = query.KIRP_normal, 
                                #save = TRUE, 
                                #save.filename = "kirpRNAseqNormal.rda", 
                                #summarizedExperiment = TRUE)

In [5]:
rownames(tumor.kirp) = rowData(tumor.kirp)$external_gene_name
tumor.kirp

class: RangedSummarizedExperiment 
dim: 56602 288 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56602): TSPAN6 TNMD ... LINC01144 AC007389.5
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(288): TCGA-HE-A5NK-01A-11R-A26U-07
  TCGA-BQ-7051-01A-12R-1965-07 ... TCGA-BQ-5882-01A-11R-1592-07
  TCGA-B1-7332-01A-11R-A32Z-07
colData names(207): barcode patient ... paper_VHL.copy..log2.normal.
  paper_NF2.copy..log2.normal.

In [6]:
ncol(tumor.kirp)

[1] 288

In [7]:
patients = colData(tumor.kirp)$patient
# patients

In [8]:
clusters = read.csv('./KN_2_i100_alpha_0.7_cluster_assignments.csv', row.names = 1)
clusters$patient = rownames(clusters)
colnames(clusters) = c( 'cluster', 'patient')
# clusters

In [9]:
common_pat = intersect(patients, clusters$patient)
clusters = clusters[common_pat, ]

In [10]:
# giving dummy cluster names for c1 as 3, and c2 as 4
clusters$cluster[clusters$cluster==1] = 3
clusters$cluster[clusters$cluster==2] = 4

# changing cluster names for c1 as 2, and c2 as 1
clusters$cluster[clusters$cluster==3] = 2
clusters$cluster[clusters$cluster==4] = 1

In [11]:
temp_pat = colnames(assay(tumor.kirp))
colnames(tumor.kirp) = sapply(temp_pat, substring ,1, 12)

In [13]:
tumor.kirp = tumor.kirp[,common_pat]

In [14]:
tumor.kirp

class: RangedSummarizedExperiment 
dim: 56602 272 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56602): TSPAN6 TNMD ... LINC01144 AC007389.5
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(272): TCGA-HE-A5NK TCGA-BQ-7051 ... TCGA-BQ-5882 TCGA-B1-7332
colData names(207): barcode patient ... paper_VHL.copy..log2.normal.
  paper_NF2.copy..log2.normal.

## DEG

In [15]:
clusters = clusters[tumor.kirp$patient, ]

In [16]:
colData(tumor.kirp)$cluster = factor(clusters$cluster,levels=c(1,2) )

In [17]:
#Design
kirp.dds <- DESeqDataSet(tumor.kirp, design = ~ cluster)

renaming the first element in assays to 'counts'

converting counts to integer mode

Warning message in DESeqDataSet(tumor.kirp, design = ~cluster):
“1452 duplicate rownames were renamed by adding numbers”


In [18]:
# Remove sample with less than 10 counts across samples for a given gene
kirp.dds = kirp.dds[rowSums(counts(kirp.dds)) > 10]

In [19]:
kirp.dds <- DESeq(kirp.dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 5711 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [20]:
res <- results(kirp.dds)

In [21]:
res

log2 fold change (MLE): cluster 2 vs 1 
Wald test p-value: cluster 2 vs 1 
DataFrame with 49011 rows and 6 columns
            baseMean log2FoldChange     lfcSE       stat      pvalue
           <numeric>      <numeric> <numeric>  <numeric>   <numeric>
TSPAN6     3248.0709      0.2604004 0.0703834   3.699742 2.15819e-04
TNMD         14.2373      0.9741640 0.2631087   3.702516 2.13472e-04
DPM1       1187.0730      0.1218389 0.0480543   2.535444 1.12305e-02
SCYL3       533.4457     -0.0216574 0.0599767  -0.361096 7.18027e-01
C1orf112    143.6480      0.4882078 0.0761791   6.408688 1.46777e-10
...              ...            ...       ...        ...         ...
AC233263.6  1.044066     0.34358453 0.2525793  1.3603039 0.173733789
HERC2P7     0.463481    -0.00642767 0.3637088 -0.0176726 0.985900055
SNORA50A.1  0.271711    -0.02752454 0.3635075 -0.0757193 0.939642414
LINC01144  92.621052    -0.33297065 0.0970281 -3.4316946 0.000599823
AC007389.5  4.938766     0.06856737 0.1935983  0.3541735 

In [22]:
# mcols(res)
summary(res)


out of 49008 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 12054, 25%
LFC < 0 (down)     : 6695, 14%
outliers [1]       : 0, 0%
low counts [2]     : 10455, 21%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [24]:
f_upgenes = which(res$padj < 0.05 & res$log2FoldChange > 1)
t_upgenes = length(f_upgenes)
f_up_res = res[f_upgenes,]

f_downgenes = which(res$padj < 0.05 & res$log2FoldChange < -1)
t_downgenes = length(f_downgenes)
f_down_res = res[f_downgenes,]

In [25]:
f_up_res

log2 fold change (MLE): cluster 2 vs 1 
Wald test p-value: cluster 2 vs 1 
DataFrame with 3371 rows and 6 columns
            baseMean log2FoldChange     lfcSE      stat      pvalue        padj
           <numeric>      <numeric> <numeric> <numeric>   <numeric>   <numeric>
KLHL13      946.0191        1.00996  0.166385   6.06999 1.27915e-09 1.88528e-08
SLC7A2      501.1772        1.72497  0.185511   9.29847 1.42482e-20 2.23422e-18
PDK4       3282.8412        1.10931  0.189747   5.84626 5.02761e-09 6.51798e-08
ABCB5        70.6046        3.34206  0.379530   8.80577 1.29953e-18 1.40743e-16
ARX          41.3995        2.90486  0.429756   6.75932 1.38641e-11 3.16917e-10
...              ...            ...       ...       ...         ...         ...
LINC00706   1.163810        1.65330  0.390489   4.23391 2.29662e-05 1.26263e-04
DNAJB5-DT   2.993867        1.45453  0.269727   5.39260 6.94465e-08 7.02224e-07
LINC00895   0.624715        2.11811  0.522034   4.05742 4.96182e-05 2.50666e-04
AP0008

In [26]:
f_down_res

log2 fold change (MLE): cluster 2 vs 1 
Wald test p-value: cluster 2 vs 1 
DataFrame with 642 rows and 6 columns
            baseMean log2FoldChange     lfcSE      stat      pvalue        padj
           <numeric>      <numeric> <numeric> <numeric>   <numeric>   <numeric>
SLC13A2      454.443       -1.68202  0.281572  -5.97369 2.31950e-09 3.22622e-08
DLEC1        177.584       -1.48061  0.195466  -7.57475 3.59827e-14 1.50963e-12
PLEKHG6      499.697       -1.07940  0.195024  -5.53470 3.11763e-08 3.39175e-07
MLXIPL      2583.983       -1.23065  0.193269  -6.36759 1.92022e-10 3.45800e-09
IL20RA       176.319       -1.00919  0.286571  -3.52160 4.28948e-04 1.69457e-03
...              ...            ...       ...       ...         ...         ...
PCAT14      4.742148       -1.12463  0.328856  -3.41984 6.26583e-04 2.36477e-03
AL353747.4  0.481624       -1.12266  0.440482  -2.54870 1.08126e-02 2.80488e-02
FOXCUT     42.565408       -1.46649  0.190056  -7.71609 1.19954e-14 5.53884e-13
AC00515

In [28]:
str(f_down_res)

Formal class 'DESeqResults' [package "DESeq2"] with 7 slots
  ..@ priorInfo      : list()
  ..@ rownames       : chr [1:642] "SLC13A2" "DLEC1" "PLEKHG6" "MLXIPL" ...
  ..@ nrows          : int 642
  ..@ listData       :List of 6
  .. ..$ baseMean      : num [1:642] 454 178 500 2584 176 ...
  .. ..$ log2FoldChange: num [1:642] -1.68 -1.48 -1.08 -1.23 -1.01 ...
  .. ..$ lfcSE         : num [1:642] 0.282 0.195 0.195 0.193 0.287 ...
  .. ..$ stat          : num [1:642] -5.97 -7.57 -5.53 -6.37 -3.52 ...
  .. ..$ pvalue        : num [1:642] 2.32e-09 3.60e-14 3.12e-08 1.92e-10 4.29e-04 ...
  .. ..$ padj          : num [1:642] 3.23e-08 1.51e-12 3.39e-07 3.46e-09 1.69e-03 ...
  ..@ elementType    : chr "ANY"
  ..@ elementMetadata:Formal class 'DFrame' [package "S4Vectors"] with 6 slots
  .. .. ..@ rownames       : NULL
  .. .. ..@ nrows          : int 6
  .. .. ..@ listData       :List of 2
  .. .. .. ..$ type       : chr [1:6] "intermediate" "results" "results" "results" ...
  .. .. .. ..$ des

In [34]:
f_up_res_sorted = f_up_res[order(f_up_res$log2FoldChange, decreasing = TRUE),]
f_down_res_sorted = f_down_res[order(f_down_res$log2FoldChange),]

In [36]:
f_down_res_sorted

log2 fold change (MLE): cluster 2 vs 1 
Wald test p-value: cluster 2 vs 1 
DataFrame with 642 rows and 6 columns
             baseMean log2FoldChange     lfcSE      stat      pvalue
            <numeric>      <numeric> <numeric> <numeric>   <numeric>
AGR3        147.26388       -3.77060  0.353689 -10.66078 1.55287e-26
CPN2        296.74607       -3.29504  0.416706  -7.90734 2.62945e-15
PAGE1         2.53225       -3.18582  0.958415  -3.32405 8.87204e-04
C1orf87      36.67659       -2.92213  0.370636  -7.88410 3.16809e-15
MUC2         11.02630       -2.74682  0.423750  -6.48217 9.04136e-11
...               ...            ...       ...       ...         ...
AC127024.6  16.524291       -1.00408  0.164698  -6.09648 1.08429e-09
ADAMTS13   609.687701       -1.00361  0.120628  -8.31991 8.80294e-17
ZNF577     440.647064       -1.00146  0.160713  -6.23138 4.62331e-10
AL121839.2 125.551993       -1.00081  0.137350  -7.28661 3.17848e-13
GPC6-AS2     0.952965       -1.00020  0.318185  -3.14347 1.

In [37]:
write.csv(f_up_res_sorted,'upgenes_poorsurv_kirp.csv')
write.csv(f_down_res_sorted,'downgenes_poorsurv_kirp.csv')